## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

In [1]:
!pip install gensim

### 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
# progress bar
from tqdm.notebook import tqdm

# Mecab, Okt 등 형태소 분석기 불러오기
import MeCab
from konlpy.tag import Okt

# 특수문자
import string

# 경고 알림 제거를 위한 라이브러리
import warnings

# gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다.
from gensim import corpora
from gensim import models

import numpy as np
import re
import pickle
import matplotlib.pyplot as pltb
# %matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

D:\anaconda\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import re

mecab = MeCab.Tagger()

def mecab_pos(text):
    pos = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정
    pos = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    return pos

In [3]:
def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    for token in temp:
        if token[1] == "NNG" or token[1]=="NNP" or token[1]=="NNB" or token[1]=="NNBC" or token[1]=="NP" or token[1]=="NR":
            nouns.append(token[0]) 
    return nouns

In [4]:
# mecab_morphs 도 작성
def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정
    temp = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    # 추출한 토큰 중에 문자열만 선택
    for token in temp:
        morphs.append(token[0])  
    
    
    return morphs

### 텍스트 전처리 함수 만들기

In [7]:
def read_documents(input_file_name):
    
    corpus = []
    
    # pk 파일을 읽어서 리스트로 변환하여 돌려줌.
    with open(input_file_name,'rb') as f:
        temp_corpus = pickle.load(f)
        
    # 리스트안의 리스트 형태인걸 일차원 리스트로 변형    
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수를 편의를 위해 조금 변형해보자.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)
        
    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # 특수 문자를 추가해보자. 파이썬의 특수문자 라이브러리를 이용해서 추가
    for i in string.punctuation:
        SW.add(i)
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding="utf-8") as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두 가지를 선택할 수 있게 만들어주는 함수를 만들어보자.
     # tqdm을 사용하여 진행 과정을 볼 수 있게 만들어보자.
    
    token_corpus = []
    
    if tokenizer == "noun":  # 명사추출
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)
            
            
    elif tokenizer == "morph":
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)
    
    elif tokenizer == "word":
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)

    return token_corpus

# 함수를 불러오는 (메인) 코드.
input_file_name = "data/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "morph" or "word"

Preprocessing:   0%|          | 0/7 [00:00<?, ?it/s]

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [8]:
# 결과 확인.
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '기업', '은행', '창업', '육성', '프로그램', '창공', '구로', '육성', '기업', '운영', '사인', '액셀러레이터', '테크', '대표', '전화', '육성', '에듀', '테크', '스타트업', '지니', '로봇', '대표', '이은승', '최근', '기업', '은행', '신용보증기금', '규모', '프리', '시리즈', '투자', '유치', '완료', '로봇', '양방향', '다자간', '화상', '교육', '플랫', '코딩', '인공지능', '교육', '가능', '올인원', '교육', '로봇', '지니', '개발', '제조', '판매', '에듀', '테크', '스타트업', '로봇', '기업', '은행', '신용보증기금', '규모', '프리', '시리즈', '투자', '유치', '완료', '테크', '제공', '지니', '언플러그드', '카드', '코딩', '전용', '엔트리', '스크래치', '파이썬', '활용', '가능', '환경', '선생', '학생', '양방향', '평가', '분석', '맞춤', '교육', '가능', '개발', '최근', '클라우드', '플랫폼', '연결', '피어', '단말기', '이용', '코딩', '교육', '제공', '시스템', '방법', '특허', '등록', '완료', '기존', '등록', '특허', '안정', '사업', '발판', '마련', '이은승', '지니', '로봇', '대표', '코로나', '수업', '시작', '학습', '격차', '학습', '양극', '현상', '현실', '학생', '공부', '환경', '개인', '맞춤', '학습', '진단', '관리', '학생', '개개인', '맞춤', '학습', '교육', '제공', '로봇', '업진', '흥원', '예비', '창업', '패키지', '지원', '사업', '시작', '중소', '벤처', '기업', '신용보증기금', '스타트업', '글로벌', '점프

### 토픽 모델링에 사용할 함수들 확인하기

In [9]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습

# 파이썬의 dict 개념이긴 하나 키 값은 숫자, value는 단어라고 생각하면 됨
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [10]:
# Dictionary 확인
print(dictionary)

Dictionary(838 unique tokens: ['가능', '개개인', '개국', '개발', '개월']...)


In [11]:
# corpus 확인
corpus[0][:5]

[(0, 3), (1, 1), (2, 1), (3, 2), (4, 1)]

In [12]:
# TFIDF 문서-단어 행렬 생성(참고)
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.04344826865943979),
 (1, 0.05035980564165338),
 (2, 0.05035980564165338),
 (3, 0.04385583331317055),
 (4, 0.03242128093073332)]

In [13]:
# LDA model 만들기
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [14]:
# LDA 결과 확인
# model.show_topic(topic_no,num_words) 

model.show_topic(0,10)  #topic 번호 0의 10개 단어보기

[('교육', 0.022510935),
 ('프로그램', 0.012872439),
 ('데이터', 0.011067237),
 ('인공지능', 0.010943264),
 ('기업', 0.009649448),
 ('개발자', 0.009471001),
 ('기술', 0.008038083),
 ('전공', 0.0078415135),
 ('분석', 0.0074260877),
 ('취업', 0.007276602)]

### 토픽 모델링을 추가하여 코드 완성하기

In [15]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3
NUM_TOPIC_WORDS = 30

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
    
    
    return corpus,dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")

    for topic_no in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_no,NUM_TOPIC_WORDS)
    
        print("Topic NO : {}".format(topic_no))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word,prob))
            
        print("\n")    
    

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus,num_topics=NUM_TOPICS,id2word=dictionary,alpha="auto",eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic NO : 0
	교육	0.01896747760474682
	데이터	0.012410718016326427
	기업	0.010856402106583118
	인텔	0.00949882622808218
	기술	0.008013228885829449
	프로그램	0.007087402045726776
	프로세서	0.0069261821918189526
	내용	0.006854703184217215
	슈퍼컴퓨터	0.0068323248997330666
	분석	0.0066976118832826614
	지원	0.006518838461488485
	과정	0.006288677453994751
	개발자	0.0062461975030601025
	학습	0.005781793501228094
	파이썬	0.0057164630852639675
	인공지능	0.005590678658336401
	전공	0.00547241885215044
	시스템	0.005454821977764368
	기반	0.005339246243238449
	플레이어	0.005036835093051195
	시장	0.005031670443713665
	활용	0.00495681818574667
	센터	0.004802829120308161
	취업	0.0047770775854587555
	분야	0.004742316901683807
	로봇	0.004719397518783808
	본문	0.004460582509636879
	올해	0.00443461025133729
	코딩	0.004377189092338085
	에픽	0.004263618029654026


Topic NO : 1
	교육	0.021540027111768723
	개발자	0.015676790848374367
	과정	0.01189704891294241
	기업	0.011338331736624241
	전공	0.010427317582070827
	취업	0.009502006694674492
	

### pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [16]:
!pip install pyldavis

  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyldavis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=5baa08838ee6b746aece016f

In [19]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model,corpus,dictionary)
data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.005975  0.009926       1        1  36.851318
1      0.018154 -0.002030       2        1  35.649057
2     -0.012179 -0.007896       3        1  27.499625, topic_info=      Term       Freq      Total Category  logprob  loglift
152    개발자  19.000000  19.000000  Default  30.0000  30.0000
130   프로그램  20.000000  20.000000  Default  29.0000  29.0000
560     인텔  13.000000  13.000000  Default  28.0000  28.0000
155     과정  16.000000  16.000000  Default  27.0000  27.0000
42      분석   8.000000   8.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
165    데이터   3.068591  19.126258   Topic3  -5.2930  -0.5388
148     활용   2.376499   9.462794   Topic3  -5.5486  -0.0907
531  슈퍼컴퓨터   2.403407  10.206385   Topic3  -5.5374  -0.1551
155     과정   2.405732  16.965927   Topic3  -5.5364  -0.6624
152    개발자   2.447608  19.964372   Topic3  -5.5192  -0.8078

[222 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.586662   가능
0         2  0.195554   가능
0         3  0.195554   가능
152       1  0.250446  개발자
152       2  0.601071  개발자
...     ...       ...  ...
360       2  0.465971   확충
360       3  0.465971   확충
148       1  0.422708   활용
148       2  0.317031   활용
148       3  0.211354   활용

[370 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])